In [1]:
import pandas as pd

In [2]:
# https://www.youtube.com/watch?v=tUh0mgB0QuA
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [3]:
#import nltk
#nltk.download('stopwords')

In [4]:
df = pd.read_csv("uuite_lexicon_nltk_sastrawi.csv")

In [5]:
temp_detokenize = []

def detokenize(text):
    text1 = text.replace(']','').replace('[','')
    arr = text1.replace('"','').replace("\'","").split(",")
    return(TreebankWordDetokenizer().detokenize(arr))

df['tweet'] = df['tweet'].astype('U').apply(detokenize)

In [6]:
df

,tweet,sentimen
0,indonesia menyelamatkan abu janda terjerah...,Negatif
1,makassar terkini jokowi revisi uu ite ro...,Negatif
2,uu ite zaman pra sejarah orang pintar h...,Positif
3,uu ite pasal pasal karet dicabut pasal a...,Negatif
4,revisi uu ite modus khawatir revisi buzz...,Negatif
...,...,...
10036,didu jk dinsyamsuddin bedakan kritikhoaxny...,Positif
10037,jepang orang terlukakalo indonesia ngak b...,Negatif
10038,wkwk ntaps ditegasin males giliran gini ...,Positif
10039,demokrat jokowi dikritik uu ite revisi,Negatif


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


In [8]:
vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
response = vectorizer.fit_transform(df.tweet)
response

<10041x17417 sparse matrix of type '<class 'numpy.float64'>'
	with 118433 stored elements in Compressed Sparse Row format>

In [9]:
#print(response)

In [10]:
vocab = vectorizer.get_feature_names()

C:\Users\User\.conda\envs\datamining\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
len(vocab)

17417

## Latent Dirichlet Allocation (LDA)


In [13]:
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
def get_topic(model):
  return [[vocab[idx] for idx in reversed(comp.argsort()[-5:]) if vocab[idx].isalnum()]
          for comp in model.components_]

In [15]:
lda2 = LatentDirichletAllocation(n_components=3, max_iter = 50, random_state=42)
lda_matrix = lda2.fit_transform(response)

In [16]:
get_topic(lda2)

[['uu', 'ite', 'revisi', 'orang', 'kritik'],
 ['lindungirezim', 'tertawa', 'dilaporin', 'ite', 'uu'],
 ['uu', 'revisi', 'ite', 'pasal', 'jokowi']]

In [17]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda2, response, vectorizer)

C:\Users\User\.conda\envs\datamining\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\User\.conda\envs\datamining\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.092226  0.066331       1        1  50.237438
2      0.017516 -0.105272       2        1  35.717967
1     -0.109742  0.038941       3        1  14.044595, topic_info=                Term        Freq       Total Category  logprob  loglift
13863         revisi  270.000000  270.000000  Default  30.0000  30.0000
4270             dpr   76.000000   76.000000  Default  29.0000  29.0000
7005        keadilan   70.000000   70.000000  Default  28.0000  28.0000
13328       presiden  109.000000  109.000000  Default  27.0000  27.0000
11964          pasal  181.000000  181.000000  Default  26.0000  26.0000
...              ...         ...         ...      ...      ...      ...
8589   lindungirezim   21.725688   87.401876   Topic3  -5.2669   0.5709
14732        senjata    3.795494   14.884553   Topic3  -7.0115   0.5964
6136             ite    6.615908  404.602331   Topic3  -6.4559  -2.1505
16883             uu    6.516522  427.320950   Topic3  -6.4710  -2.2203
7376            kena    4.517994  100.286384   Topic3  -6.8373  -1.1370

[181 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
36         1  0.423929        abu
36         2  0.588791        abu
169        3  0.754428    agustus
241        3  0.818357  akademisi
325        1  0.973807       alat
...      ...       ...        ...
16883      2  0.446971         uu
16883      3  0.016381         uu
16990      2  0.975476     wacana
17037      2  0.988909      wakil
17149      2  0.968899     widodo

[184 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])